In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoModel, AutoTokenizer
from transformers import BertConfig, BertModel

In [2]:
subject_df = pd.read_csv('../data/subject_meta.csv')
subject_df.index = subject_df['subject_id']
# subject_df

In [12]:
question_df = pd.read_csv('../data/question_meta.csv')
question_df['name'] = question_df['subject_id']
for i in range(len(question_df)):
    row = question_df.iloc[i]
    subjects =  [int(x) for x in row['subject_id'].strip('[]').split(',')]
    all_sbujects = [subject_df.loc[x]['name'] for x in subjects]
    question_df['name'].iloc[i] = ', '.join(all_sbujects)

question_df

C:\Users\loveb\AppData\Local\Temp\ipykernel_44780\1459812776.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  question_df['name'].iloc[i] = ', '.join(all_sbujects)


,question_id,subject_id,name
0,716,"[0, 1, 5, 98, 147, 167]","Maths, Number, Decimals, Basic Arithmetic, Men..."
1,343,"[0, 1, 8, 173]","Maths, Number, Fractions, Adding and Subtracti..."
2,1681,"[0, 1, 12, 176]","Maths, Number, Percentages, Percentages of an ..."
3,834,"[0, 1, 12, 177]","Maths, Number, Percentages, Percentage Increas..."
4,840,"[0, 17, 29, 116]","Maths, Algebra, Sequences, Linear Sequences (n..."
...,...,...,...
1769,1021,"[0, 17, 31, 104]","Maths, Algebra, Simultaneous Equations, Solvin..."
1770,1673,"[0, 1, 98, 350]","Maths, Number, Basic Arithmetic, Basic Arithme..."
1771,1575,"[0, 1, 98, 350]","Maths, Number, Basic Arithmetic, Basic Arithme..."
1772,227,"[0, 1, 98, 350]","Maths, Number, Basic Arithmetic, Basic Arithme..."


In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
config = BertConfig.from_pretrained('bert-base-cased', num_hidden_layers=4)
bert_model = AutoModel.from_pretrained("bert-base-cased",  config=config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.layer.5.attention.self.value.weight', 'bert.encoder.layer.9.attention.output.dense.bias', 'bert.encoder.layer.10.attention.self.value.weight', 'bert.encoder.layer.7.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.self.key.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'bert.encoder.layer.11.attention.self.value.weight', 'bert.encoder.layer.10.output.LayerNorm.bias', 'bert.encoder.layer.8.attention.self.key.bias', 'cls.predictions.bias', 'bert.encoder.layer.4.output.LayerNorm.weight', 'bert.encoder.layer.7.attention.output.dense.bias', 'bert.encoder.layer.9.output.dense.weight', 'cls.seq_relationship.bias', 'bert.encoder.layer.5.attention.output.dense.bias', 'bert.encoder.layer.6.output.LayerNorm.weight', 'bert.encoder.layer.8.attention.output.Lay

In [8]:
sentences = question_df['name'].to_list()
sentences

['Maths, Number, Decimals, Basic Arithmetic, Mental Multiplication and Division, Multiplying and Dividing with Decimals',
 'Maths, Number, Fractions, Adding and Subtracting Fractions',
 'Maths, Number, Percentages, Percentages of an Amount',
 'Maths, Number, Percentages, Percentage Increase and Decrease',
 'Maths, Algebra, Sequences, Linear Sequences (nth term)',
 'Maths, Algebra, Linear Equations, Solving Equations',
 'Maths, Geometry and Measure, Types, Naming and Estimating, Angles',
 'Maths, Geometry and Measure, Compound Measures, Speed, Distance, Time',
 'Maths, Geometry and Measure, Units of Measurement, Time',
 'Maths, Data and Statistics, Box Plots, Pie Chart, Data Representation',
 'Maths, Data and Statistics, Scatter Diagram, Data Representation',
 'Maths, Data and Statistics, Stem and Leaf Diagram, Data Processing, Range and Interquartile Range from a List of Data, Data Representation',
 'Maths, Algebra, Sequences, Linear Sequences (nth term)',
 'Maths, Algebra, Linear Equa

In [9]:
tokenized = tokenizer(sentences,
          padding="max_length", max_length=64)

In [10]:
bert_with_classifier_head = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
input_ids = torch.tensor(tokenized['input_ids'])
output = bert_model.forward(input_ids)
output['last_hidden_state'].size()

torch.Size([1774, 64, 768])

In [13]:
vectors = output['last_hidden_state'].detach().numpy()
rslt = vectors.reshape(vectors.shape[0], vectors.shape[1] * vectors.shape[2])
print(rslt.shape)

(1774, 49152)


In [14]:
# np.save('../data/extracted_feats.npy', rslt)
np.save('../data/extracted_feats_question.npy', rslt)

In [18]:
print(vectors.shape)
cat = np.sum(vectors, axis=1)
print(cat.shape)
np.save('../data/extracted_feats_question_comp.npy', cat)

(1774, 64, 768)
(1774, 768)


In [19]:
cat

array([[ -1.4207188,  11.350973 ,  24.057917 , ..., -22.311497 ,
         39.141266 , -28.604828 ],
       [-10.238108 ,   6.1664276,  41.76565  , ..., -34.864746 ,
         50.953007 , -39.82339  ],
       [-20.923878 ,  11.010177 ,  40.471207 , ..., -32.72176  ,
         47.281593 , -40.54172  ],
       ...,
       [-15.899199 ,   7.9688454,  40.233932 , ..., -36.25874  ,
         44.254143 , -40.311874 ],
       [-15.899199 ,   7.9688454,  40.233932 , ..., -36.25874  ,
         44.254143 , -40.311874 ],
       [-15.899199 ,   7.9688444,  40.233932 , ..., -36.258743 ,
         44.254147 , -40.31187  ]], dtype=float32)